In [1]:
import torch
import pandas as pd
from torch import nn, optim
from torch.utils import data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [3]:
columns = [
    'Mean_IP',          'SD_IP',            'Kurtosis_IP',
    'Skewness_IP',      'Mean_DMSNR',       'SD_DMSNR',
    'Kurtosis_DMSNR',   'Skewness_DMSNR',   'Class'
]

In [4]:
dataset = pd.read_csv('../archive/HTRU_2.csv', header=None, names=columns)

In [5]:
y = dataset['Class']
x = dataset.drop(columns=['Class'])

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [7]:
scaler = MinMaxScaler()

In [8]:
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [9]:
class CatSpaceModel(nn.Module):
    def __init__(self, input_dim):
        super(CatSpaceModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, 64)
        self.layer2 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, n):
        n = self.relu(self.layer1(n))
        n = self.relu(self.layer2(n))
        return self.output_layer(n)

In [10]:
def train_model(model, feats, targets, epochs=100, batch_size=32):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    feats_tensor = torch.FloatTensor(feats)
    targets_tensor = torch.FloatTensor(targets.values).view(-1, 1)

    data_set = data.TensorDataset(feats_tensor, targets_tensor)
    dataloader = data.DataLoader(data_set, batch_size=batch_size, shuffle=True)

    model.train()

    for epoch in range(epochs):
        epoch_loss = 0

        for inputs, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        epoch_loss /= len(dataloader)

        if (epoch+1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")
    return model

In [11]:
def evaluate_model(model, feats, targets):
    model.eval()
    with torch.no_grad():
        feats_tensor = torch.FloatTensor(feats)
        targets_tensor = torch.FloatTensor(targets.values).view(-1, 1)

        outputs = model(feats_tensor)
        predicted = (outputs >= 0.5).float()

        accuracy = (predicted.eq(targets_tensor).sum().item()) / len(targets_tensor)
        print(f'Accuracy: {accuracy * 100:.2f}%')

In [12]:
def test_model(model, feats, targets):
    model.eval()
    with torch.no_grad():
        feats_tensor = torch.FloatTensor(feats)
        targets_tensor = torch.FloatTensor(targets.values).view(-1, 1)

        outputs = model(feats_tensor)
        predicted = (outputs >= 0.5).float()

        cm = confusion_matrix(targets_tensor, predicted)
        cr = classification_report(targets_tensor, predicted)
        print(f"Confusion Matrix:\n{cm}")
        print()
        print(f"Classification Report:\n{cr}")

In [13]:
cat_space = CatSpaceModel(input_dim=x_train.shape[1])

In [14]:
cat_space = train_model(cat_space, x_train, y_train)

Epoch [5/100], Loss: 0.0808
Epoch [10/100], Loss: 0.0772
Epoch [15/100], Loss: 0.0750
Epoch [20/100], Loss: 0.0720
Epoch [25/100], Loss: 0.0709
Epoch [30/100], Loss: 0.0693
Epoch [35/100], Loss: 0.0694
Epoch [40/100], Loss: 0.0694
Epoch [45/100], Loss: 0.0679
Epoch [50/100], Loss: 0.0676
Epoch [55/100], Loss: 0.0674
Epoch [60/100], Loss: 0.0668
Epoch [65/100], Loss: 0.0667
Epoch [70/100], Loss: 0.0664
Epoch [75/100], Loss: 0.0661
Epoch [80/100], Loss: 0.0664
Epoch [85/100], Loss: 0.0659
Epoch [90/100], Loss: 0.0654
Epoch [95/100], Loss: 0.0650
Epoch [100/100], Loss: 0.0652


In [15]:
evaluate_model(cat_space, x_test, y_test)

Accuracy: 98.10%


In [16]:
test_model(cat_space, x_test, y_test)

Confusion Matrix:
[[3239   20]
 [  48  273]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      3259
         1.0       0.93      0.85      0.89       321

    accuracy                           0.98      3580
   macro avg       0.96      0.92      0.94      3580
weighted avg       0.98      0.98      0.98      3580



In [17]:
torch.save(cat_space.state_dict(), '../models/cat_space.pth')